In [3]:
pip install numpy

In [4]:
pip install catboost

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

# 데이터셋 불러오기
df = pd.read_csv('new_df.csv')

# 특성과 레이블 분리
X = df['preprocessed_text']  # 'text_column'을 실제 텍스트 열 이름으로 변경하세요
y = df['label']  # 'label_column'을 실제 레이블 열 이름으로 변경하세요

# 데이터셋을 훈련, 검증, 테스트 세트로 분할
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# TF-IDF를 사용하여 텍스트 데이터 벡터화
vectorizer = TfidfVectorizer(max_features=30)
X_train_vec = vectorizer.fit_transform(X_train)
X_valid_vec = vectorizer.transform(X_valid)
X_test_vec = vectorizer.transform(X_test)

In [6]:
# CatBoost 모델 훈련 및 평가
catboost_model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=0, random_state=42)
catboost_model.fit(X_train_vec, y_train, eval_set=(X_valid_vec, y_valid), early_stopping_rounds=50)

In [7]:

# 모델 평가
y_train_pred = catboost_model.predict(X_train_vec)
y_valid_pred = catboost_model.predict(X_valid_vec)
y_test_pred = catboost_model.predict(X_test_vec)

train_accuracy = accuracy_score(y_train, y_train_pred)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f'훈련 정확도: {train_accuracy:.3f}')
print(f'검증 정확도: {valid_accuracy:.3f}')
print(f'테스트 정확도: {test_accuracy:.3f}')

# 검증 세트에 대한 분류 리포트 출력
print("\n검증 세트 분류 리포트:\n")
print(classification_report(y_valid, y_valid_pred))

# 테스트 세트에 대한 분류 리포트 출력
print("\n테스트 세트 분류 리포트:\n")
print(classification_report(y_test, y_test_pred))

훈련 정확도: 0.743
검증 정확도: 0.693
테스트 정확도: 0.688

검증 세트 분류 리포트:

              precision    recall  f1-score   support

           2       0.50      0.53      0.51       498
           4       0.82      0.85      0.84       400
           8       0.48      0.57      0.52       247
           9       0.82      0.94      0.88       235
          10       0.54      0.61      0.57       168
          13       0.81      0.81      0.81       150
          18       0.65      0.41      0.51       162
          30       0.93      0.97      0.95       433
          31       0.63      0.58      0.61       318
          32       0.66      0.51      0.58       313
          33       0.83      0.38      0.52       449
          34       0.71      0.97      0.82       616
          35       0.61      0.62      0.62       379

    accuracy                           0.69      4368
   macro avg       0.69      0.67      0.67      4368
weighted avg       0.70      0.69      0.68      4368


테스트 세트 분류 리포트:

   

In [8]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
param_grid = {
    'iterations': [500],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8]
}

catboost_tuned = CatBoostClassifier(loss_function='MultiClass', verbose=0, random_state=42)
grid_search = GridSearchCV(estimator=catboost_tuned, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
grid_search.fit(X_train_vec, y_train)

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x000002968F9F5450>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'iterations': [500],
                         'learning_rate': [0.01, 0.1, 0.2]},
             scoring='accuracy')

In [9]:
# 최적의 하이퍼파라미터로 다시 모델 훈련 및 평가
best_params = grid_search.best_params_
print(f"\n최적의 하이퍼파라미터: {best_params}")


최적의 하이퍼파라미터: {'depth': 8, 'iterations': 500, 'learning_rate': 0.2}


In [10]:
best_catboost_model = CatBoostClassifier(**best_params, random_state=42, verbose=0)
best_catboost_model.fit(X_train_vec, y_train, eval_set=(X_valid_vec, y_valid), early_stopping_rounds=50)

In [11]:
# 모델 평가 (최적의 하이퍼파라미터 사용)
y_train_pred = best_catboost_model.predict(X_train_vec)
y_valid_pred = best_catboost_model.predict(X_valid_vec)
y_test_pred = best_catboost_model.predict(X_test_vec)

train_accuracy = accuracy_score(y_train, y_train_pred)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f'최적 모델 훈련 정확도: {train_accuracy:.3f}')
print(f'최적 모델 검증 정확도: {valid_accuracy:.3f}')
print(f'최적 모델 테스트 정확도: {test_accuracy:.3f}')

# 검증 세트에 대한 분류 리포트 출력 (최적 모델)
print("\n최적 모델 검증 세트 분류 리포트:\n")
print(classification_report(y_valid, y_valid_pred))

# 테스트 세트에 대한 분류 리포트 출력 (최적 모델)
print("\n최적 모델 테스트 세트 분류 리포트:\n")
print(classification_report(y_test, y_test_pred))


최적 모델 훈련 정확도: 0.784
최적 모델 검증 정확도: 0.709
최적 모델 테스트 정확도: 0.706

최적 모델 검증 세트 분류 리포트:

              precision    recall  f1-score   support

           2       0.53      0.54      0.54       498
           4       0.84      0.86      0.85       400
           8       0.50      0.55      0.53       247
           9       0.82      0.92      0.87       235
          10       0.55      0.64      0.59       168
          13       0.84      0.85      0.84       150
          18       0.69      0.48      0.56       162
          30       0.95      0.97      0.96       433
          31       0.65      0.65      0.65       318
          32       0.66      0.57      0.61       313
          33       0.85      0.39      0.54       449
          34       0.71      0.98      0.82       616
          35       0.62      0.62      0.62       379

    accuracy                           0.71      4368
   macro avg       0.71      0.69      0.69      4368
weighted avg       0.72      0.71      0.70      43